In [ ]:
# computing the two body density while the simulation is running
# ... increases simulation time 
COMPUTE_TWO_BODY_DENSITY = True

# if set to true, notebook will dump data in XYZ format to be opened ...
# ... in e.g. MD-viewers like Ovito
DUMP_DATA = False

Thinking about Lennard-Jones interacting particles as having a hard core starting at the point where $u(r)=0$, then, writing down the interaction potential $$u(r)=4\varepsilon\left(\frac{\sigma}{r}\right)^6\left(\left(\frac{\sigma}{r}\right)^6-1\right)$$ from which can be immediately read off, that the root lies at $r=\sigma$. Then if there are $N$ particles in the system, the packing fraction $$\eta=\frac{4}{3}\pi\left(\frac{\sigma}{2}\right)^3\cdot N/V$$ or more generally in $d$ spatial dimensions $$\eta=\frac{\pi^{d/2}}{\Gamma\left(\frac{d}{2}+1\right)}\cdot\left(\frac{\sigma}{2}\right)^d\cdot N/V$$ would for a system of non-overlapping particles be the fraction of the volume taken up by the particles.

In [ ]:
import numpy as np
from scipy.special import gamma

import matplotlib.pyplot as plt

In [ ]:
# simulation dimensions
N = 50

# spatial dimensions
d = 3

# box length for PBC
box_length = 1

# volume and number density
V = box_length**d
rho = N / V

In [ ]:
def xyz_snapshot_dump(filename, pos, comment='', element='H', opening_type='a'):

    # File format according to https://en.wikipedia.org/wiki/XYZ_file_format
    # <number of atoms>
    # comment line
    # <element> <X> <Y> <Z>

    with open(filename, opening_type, encoding='ascii') as file:

        # dumping number of atoms
        file.write(str(N)+'\n')

        # comment line
        file.write(comment+'\n')
        
        # position dump
        for i in range(N):
            x, y, z = pos[i][0], pos[i][1], pos[i][2]
            file.write('%s %f %f %f\n' % (element, x, y, z))

In [ ]:
# temperature given energy scale
kBT = 1

# particle diameter and Lennard-Jones energy scale in units of kBT
eta = 0.6
epsilon = 1 * kBT

# computing sigma from packing fraction
sigma = 2 * (gamma(1+d/2) * (eta/rho) / np.pi**(d/2))**(1/d)

In [ ]:
print('η = %.2f => 𝜎 = %.2f, ε/kT = %.2f' % (eta, sigma, epsilon/kBT))

In [ ]:
# interaction energy
def u(r):
    # Lennard-Jones-Potential
    return 4*epsilon * (sigma / r)**6 * ((sigma / r)**6 - 1)

I will employ periodic boundary conditions, to create conditions as close as possible to the bulk fluid. To achieve this, two things have to be done:
1. When computing potential energy of any given particle, this has to be done with including as many neighboring boxes as the reach of the potential requires. For Lennard-Jones it can be for all intents and purposes argued, that it's reach is about $2\sigma$, where $\sigma$ is chosen an order of magnitude smaller than the unit cell diameter. This way, only the immediately surrounding 26 other unit cells have to also be taken into account
2. When updating positions, they have to be taken modulo any lattice vector $\vec{R}=\sum_jR^j\vec{a}_j$ where $R^j\in\mathbb{Z}$

In [ ]:
# surrounding box lattice unit vectors, ...
# ... 26 (surrounding copies) + 1 (original)
lattice_vectors = [
    box_length * np.array([i,j,k])
    for i in [-1,0,1]
    for j in [-1,0,1]
    for k in [-1,0,1]
]

In [ ]:
# indexing the shape of the positions array
NUM, DIM  = 0, 1

# drawing random positions in a box
positions = box_length * np.random.rand(N, d)

We will now try to get the Monte-Carlo Simulation into a state where it samples according to a canonical Boltzmann-distribution $$\frac{e^{-\beta H(\{\vec{x}_k,\vec{p}_k\})}}{Z}$$ where $H$ is total system energy, $\beta=1/k_{\mathrm{B}}T$ and $$Z(T)=\int\frac{1}{N!}\left(\frac{\mathrm{d}^dx_1\,\mathrm{d}^dp_1}{h^d}\right)\cdots\left(\frac{\mathrm{d}^dx_N\,\mathrm{d}^dp_N}{h^d}\right)e^{-\beta H(\{\vec{x}_k,\vec{p}_k\})}$$ with number of spatial dimensions $d$ and particle count $N$. For Hamiltonians of the form $$H(\{\vec{x}_k,\vec{p}_k\})=\sum_k\frac{\vec{p}_k^2}{2m_k}+\sum_k v(\vec{x}_k)+\sum_{i<j}u(|\vec{x}_i-\vec{x}_j|)$$ all velocities will be Maxwell-Boltzmann distributed ($v(\vec{r})$ is the external potential, e.g. given by hard walls of some confining box). For the Boltzmann-distribution now factors into a kinetic and potential energy part, which can be seperately integrated to the two partition function factors $Z=Z_{\mathrm{id}}Z_{\mathrm{ex}}$ with

$$Z_{\mathrm{id}}=\frac{1}{N!}\prod_k\frac{(\lambda_{\mathrm{th},k})^d}{V}$$

$$Z_{\mathrm{ex}}=\int\left(\mathrm{d}^dx_1\,\frac{e^{-\beta v(\vec{x}_1)}}{V}\right)\cdots\left(\mathrm{d}^dx_N\,\frac{e^{-\beta v(\vec{x}_N)}}{V}\right)\exp\left(-\beta\sum_{i<j}u(|\vec{x}_i-\vec{x}_j|)\right)$$

where $\lambda_{\mathrm{th},k}=h/\sqrt{2\pi\cdot m_k\cdot k_{\mathrm{B}}T}$ is the thermal de-Broglie wavelength of particle $k$ (the same for all particles iff all their masses are the same) and $V$ is the effective (in general temperature dependent) volume $$V=\int\mathrm{d}^dr\,e^{-\beta v(\vec{r})}$$ Now looking at the ensemble average of the velocity $\vec{v}_k$ of particle $k$

$$\left\langle\vec{v}_k\right\rangle\propto\int\mathrm{d}^dv_k\exp\left(-\frac{\frac{1}{2}m\vec{v}_k^2}{k_{\mathrm{B}}T}\right)\,\vec{v}_k$$

This result is arrived at by first integrating out the position dependence to a factor of $V^N\cdot Z_{\mathrm{ex}}$, followed by carrying out the $N-1$ momentum integrals that do not depend on $\vec{v}_k$, only to finally cancel all of those out with factors appearing in $Z=Z_{\mathrm{id}}Z_{\mathrm{ex}}$. Since velocities will apparently always be Maxwell-Boltzmann distributed, the only interesting distribution to sample is the one of the spatial configurations of particles. Thus the steps drawn will be a change of system configuration, with the acceptance probability $$p_{1\to 2}=\min\left(1,e^{-\beta(\Delta U_{1\to 2})}\right)$$ where $\Delta U_{1\to 2}$ is the change in potential energy due to changing the system configuration from $1$ to $2$.

As a final remark, a motivation for the definition of $V$ (for completeness sake): for the case of a hard wall surrounding a domain $\mathcal{D}$ in $d$-dimensional space, i.e. the external potential $$v_{\mathrm{\,hard\,wall}}(\vec{r})=\begin{cases}\infty&\vec{r}\notin\mathcal{D}\\0&\vec{r}\in\mathcal{D}\end{cases}$$ the Boltzmann-factor $e^{-\beta v(\vec{r})}$ vanishes outside $\mathcal{D}$ while being $1$ inside it. Thus, in the hard wall case, $V$ is exactly the volume of the domain $\mathcal{D}$ (independent of temperature!). **Note that instead of using a hard wall**, that will add boundary effects like adsorption, I will employ **periodic boundary conditions** and will be using the unit cell volume for $V$.

In [ ]:
def potential_energy_difference_due_to_step_k(k, step_k, positions):
    """ returns the potential energy difference (new minus old) that would occur if particle k from the configuration stored in the numpy.array positions were shifted by step_k """

    # getting old and new positions
    old_position_k = positions[k]
    new_position_k = old_position_k + step_k

    # getting old and new potential energy ...
    old_potential_energy = 0.0
    new_potential_energy = 0.0

    # ... by looping over every particle except for k
    for j in range(N):

        if j == k:
            continue
        
        # adding j-contribution to old and new potential ...
        # ... energy respectively for every PBC copy of ...
        # ... particle j
        for R in lattice_vectors:
                
            # current PBC copy of particle j, shifted by one of the ...
            # ... 26 possible lattice vectors like box_length * (-1, 0, 1)
            current_j_copy = positions[j] + R
            
            # old and new distance to particle j
            r_jk_old = np.linalg.norm(current_j_copy - old_position_k)
            r_jk_new = np.linalg.norm(current_j_copy - new_position_k)
            
            # adding potential energy contribution of the currently ...
            # ... considered PBC copy of particle j
            old_potential_energy += u(r_jk_old)
            new_potential_energy += u(r_jk_new)

    return new_potential_energy - old_potential_energy

In [ ]:
def draw_step_k(cov=sigma**2):
    """ returns a normal-distributed d-dimensional step meant for a single particle in the Metropolis Monte Carlo simulation """

    # choosing new position from a normal distribution
    return np.random.multivariate_normal(

        # mean
        np.zeros(d),

        # covariance (diagonal with equal entries -> isotropic)
        np.diag( cov * np.ones(d) ),

        # shape
        1

    # random.multivariate_normal returns a nested ...
    # ... result of the form [[x_1, ..., x_d]], thus ...
    # ... calling the 0-th element of this one-element ...
    # ... array yields the sought vector
    )[0]

Very important observables are the $l$-body densities $$\rho^{(l)}(\vec{r}_1,\dots,\vec{r}_l):=\left\langle\sum_{j_1<\cdots<j_l}\delta(\vec{r}_1-\vec{x}_{j_1})\cdots\delta(\vec{r}_1-\vec{x}_{j_1})\right\rangle$$ where the sum over indices has the greater than signs in order to get every unique combination of particles exactly once. $\rho^{(l)}$ is proportional to the probability density of finding $l$ (indistinguishable) particles exactly the positions $\vec{r}_1,\dots,\vec{r}_l$. Of particular interest are the one-body density $\rho^{(1)}(\vec{r})=:\rho(\vec{r})$, which is just the molar density of the gas (which is supposed to be uniformly equal to some constant $\rho$ in this simulation), and the two body density $\rho^{(2)}(\vec{r}_1,\vec{r}_2)$, from which, in principle pressure $$P=\rho\,k_{\mathrm{B}}T-\frac {1}{6}\int_0^\infty 4\pi r^2\,\mathrm{d}r\cdot r\frac{\partial u(r)}{\partial r}\cdot\rho^{(2)}(r)$$ isotermal compressibility $$k_{\mathrm{B}}T\frac{\partial\rho}{\partial P}=1+\int_0^\infty 4\pi r^2\,\mathrm{d}r\,\frac{\rho^{(2)}(r)-\rho^2}{\rho}$$ energy due to particle pair interaction $$\frac{U_{\mathrm{ex}}}{V}=\frac{1}{2}\int_0^\infty 4\pi r^2\,\mathrm{d}r\,\rho^{(2)}(r)\,u(r)$$ and many other density-related thermodynamic quantities can be computed. Note that in isotropic systems of isotropically interacting particles $\rho^{(2)}$ is radially symmetric.

The $l$-body densities are fields, which can for example be expanded in a practical ONB compatible with boundary conditions (say Poisson-Equation-eigenmodes for a hard-wall domain $\mathcal{D}$ or a Fourier-Series in reciprocal basis vectors for periodic boundary conditions) which is then cut-off beyond details not resolvable due to the discrete and finite nature of the simulation. One may also

1. introduce a $d$-dimensional grid of bins
2. draw sufficiently many configurations from a $e^{-\beta H}/Z$-distribution
3. per drawn configuration: count, for each bin, the number of particles it contains

For the radially symmetric $\rho^{(2)}(r)$ the binning may also be introduced on the separation $r$; I will do the latter.

If one wants to see $\rho^{(2)}(r)$ up to $m$ multiplies of $\sigma$, then one should consider: if there are $N$ particles in a $d$-dimensional box subject to periodic boundary conditions, then an upper bound as to how many particles one should expect to be able to pass before looping back to start due to the periodicity is about $N^{1/d}$ (motivated by imagining a cubic crystal of $N$ particles: the number of particles along any side will be $N^{1/d}$). The Lennard Jones fluid $\rho^{(2)}(r)$ vanishes for $r<\sigma$, before shooting up to a peak, after which it which oscillates with growing $r$ in a dampened manner around its $r\to\infty$ value of $\rho^2$. This dampened oscillation occurs over a distance of a few $\sigma$. If one wants to have a chance at observing for example three peaks of oscillation in three-dimensional space, then $50^{1/3}\approx 3.68$ even leaves almost one $\sigma$ of buffer to observe the desired effect (which, as a reminder, is finding the *bulk* fluid two-body density).

In [ ]:
def distances_to_particle_k(k, positions):
    """ Returns the distances that other particles have to particle k """

    # creating the array which will be binned afterwards
    distances = []

    # looping over every particle except for k
    for j in range(N):

        if j == k:
            continue

        # due to periodic boundary conditions, the surrounding ...
        # ... unit cells are also included, every lattice vector ...
        # ... shifted copy is also included. Going beyond the ...
        # ... immediately surrounding unit cells however is ...
        # ... will yield unphysical results due to the periodicity ...
        # ... anyways
        for R in lattice_vectors:

            # current PBC copy of particle j, shifted by one of the ...
            # ... 26 possible lattice vectors like box_length * (-1, 0, 1)
            current_j_copy = positions[j] + R
                
            # distance between the current PBC copy of particle j ...
            # ... and the particle k currently used as the center of ...
            # ... the two body correlation estimator
            r_jk = np.linalg.norm(current_j_copy - positions[k])

            distances.append(r_jk)
    
    return distances

In [ ]:
def two_body_correlation(particle_positions, sample_particle_count, r_max=N**(1/d)*sigma, bin_count=100):
    """ randomly picks (without repetition) sample_particle_count particles, and bins all of them into a histogram """

    if sample_particle_count > N:
        raise ValueError('sample_particle_count=%d is greater than overall particle count N=%d' % (sample_particle_count, N))

    # particle indices from which the index of a sampled ...
    # ... particle is pulled and removed
    particle_indices = list(range(0, N))

    # distances array
    distances = []

    for sample in range(sample_particle_count):
        
        # choosing a random particle
        k = particle_indices.pop(np.random.randint(len(particle_indices)))

        # appending distances of other particles to particle j. NOTE: The ...
        # ... plus sign here is python's in-built list concatenation! No numpy ...
        # ... involved 
        distances = distances + distances_to_particle_k(k, particle_positions)

    # the information contained in the histogram is the number of particles ... 
    # ... found in the r-shell around particle k, i.e. rho^2 * 4*pi*r^2*dr * rho^(2)
    histogram, r_bin_edges = np.histogram(distances, bins=bin_count, range=(0, r_max))
    r = r_bin_edges[:-1]

    # averaging the size of the bins to get a value for the shell width dr
    dr = np.mean(np.diff(r_bin_edges))

    # integrating rho^(2) yierds N * rho = N^2/V = Integral over exact ...
    # ... distance histogram / V. Rearranging renders the following ...
    # ... r-dependent conversion factor
    rho_2 = (rho / sample_particle_count) * histogram / (4*np.pi*r**2*dr)
    
    return (r, rho_2)

In [ ]:
# setting the bin count and cutoff radius for the two body density
r_max = N**(1/d)*sigma
bin_count = 100

# every step of the simulation, the two body density is computed by ...
# ... randomly sampling particles from the configuration and binning ...
# ... their distance to every other particle in the system. The ...
# ... number of randomly (without repetion) sampled particles is ...
# ... set here:
sampled_particles_per_step = N

# defining the distance space
r = np.linspace(0, r_max, bin_count)

# this rho_2 will contain the average measured two body density
rho_2 = np.zeros(bin_count)

# number of Metropolis Monte-Carlo steps
step_count = 2000

# burn in-time where simulation should be omitted
burn_in_time = int(0.1 * step_count)

In [ ]:
# user feedback about progress
from tqdm.notebook import tqdm

# time resolving the acceptance rate by having a ...
# ... string of 0 and 1, which can later be convolved ...
# ... into a rolling average of freely choosable size
acceptance_string = []

for step in tqdm(range(step_count)):

    # making every particle do a step
    for k in range(N):

        # making an isotropically drawn step of ...
        # ... covariance (f * sigma)**2, where sigma is ...
        # ... the diameter of the Lennard-Jones ...
        # ... hard core { u > 0 } 
        step_k = draw_step_k(cov=(0.1*sigma)**2)

        # energy difference
        energy_difference = potential_energy_difference_due_to_step_k(k, step_k, positions)

        # rolling whether to accept
        if np.random.rand() < np.exp(-energy_difference/kBT):

            # append 1 if accepted
            acceptance_string.append(1)

            # setting all positions of shifted particle ...
            for mu in range(d):
                # ... modulo box_length for PBC
                positions[k][mu] = (positions[k][mu] + step_k[mu]) % box_length
        
        else:
            # append 0 if rejected
            acceptance_string.append(0)

    # dumping after all particles have been looped
    if DUMP_DATA:
        xyz_snapshot_dump('N=%d_eta=%.1f_eps=%.1fkT.xyz' % (N, eta, epsilon), positions, comment='step %d' % step)

    # computing the two body density if the burn-in ...
    # ... is over
    if COMPUTE_TWO_BODY_DENSITY:
        if step >= burn_in_time:

            # adding two body density of current configuration
            r_current, rho_2_current = two_body_correlation(
                positions,
                sampled_particles_per_step,
                r_max=r_max,
                bin_count=bin_count
            )

            rho_2 = rho_2 + rho_2_current

if COMPUTE_TWO_BODY_DENSITY:
    # dividing by the number of samples
    rho_2 = rho_2 / (step_count - burn_in_time)

In [ ]:
# implementing rolling average with ones
kernel_size = int(step_count/2) * N
kernel = np.ones(kernel_size) / kernel_size

# local average of acceptances
acceptance_rates = np.convolve(np.array(acceptance_string), kernel, mode='valid')

# plotting the running-average of the acceptance rate over time
plt.plot(np.array(range(acceptance_rates.size)) / N, 100*acceptance_rates, color='black')

# labeling
plt.xlabel('simulation step')
plt.ylabel(r'acceptance rate in %')

plt.show()

In [ ]:
# In case the two body density was computed while ...
# ... simulating, draw it up against the low density ...
# ... limit
if COMPUTE_TWO_BODY_DENSITY:
    
    # simulation results
    plt.plot(r/sigma, rho_2 / rho**2, color='black', linestyle='-', label=r'simulation $\rho^{(2)}(r)$')
    
    # low density (i.e. second order in density) limit
    plt.plot(r/sigma, np.exp(-u(r)/kBT), color='black', linestyle='dashed', label=r'$\rho\to 0$ limit $\rho^2\cdot e^{-\beta u(r)}+\mathcal{O}(\rho^3)$')

    # drawing in a line of constant one, because that is the ...
    # ... r->infty value of rho^(2) / rho^2
    plt.axhline(y=1, color='black', linestyle='dotted', label=r'line of $\rho^{(2)}\overset{!}{=}\rho^2$')

    # labeling
    plt.xlabel(r'$r/\sigma$')
    plt.ylabel(r'$\rho^{(2)}(r)\;/\;\rho^2$')
    plt.title('Structure of a Lennard-Jones (bulk) fluid taken\nfrom a Metropolis-Monte-Carlo simulation\n'+r'at $\eta=%.1f$ and $\varepsilon/k_{\mathrm{B}}T=%.1f$' % (eta, epsilon/kBT))

    # legend for the user
    plt.legend()

    # saving the figure
    plt.savefig('LJ_two_body_density_at_N=%d_eta=%.1f_eps=%.1fkBT.pdf' % (N, eta, epsilon))

    plt.show()